In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../data/raw/insurance_data.txt", sep = "|")

C:\Users\ltlid\AppData\Local\Temp\ipykernel_9440\3591821996.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/insurance_data.txt", sep = "|")


In [3]:
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [4]:

# Clean date column
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])

In [5]:
# Calculate metrics
df['HasClaim'] = df['TotalClaims'] > 0
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

In [11]:
# Null Hypothesis 1
# H₀: There are no risk differences across provinces
# We’ll test for Claim Frequency and Claim Severity across provinces.
# Claim Frequency: Chi-squared test

# Frequency Table: Province vs HasClaim
freq_table = pd.crosstab(df['Province'], df['HasClaim'])
chi2, p, dof, expected = stats.chi2_contingency(freq_table)

print(f"Chi-squared test for Claim Frequency by Province: p-value = {p:.4f}")

# Claim Severity: ANOVA (if normal), else Kruskal-Wallis
# Subset only rows where there is a claim
claim_data = df[df['HasClaim']]

# Grouped by Province
grouped_severity = [group['TotalClaims'].dropna() for _, group in claim_data.groupby('Province')]

# Kruskal-Wallis test (non-parametric ANOVA)
h_stat, p_val = stats.kruskal(*grouped_severity)

print(f"Kruskal-Wallis test for Claim Severity by Province: p-value = {p_val:.4f}")

Chi-squared test for Claim Frequency by Province: p-value = 0.0000
Kruskal-Wallis test for Claim Severity by Province: p-value = 0.0000


Kruskal-Wallis test for Claim Severity by Province: p-value = 0.0000


In [ ]:
# Null Hypothesis 2
# H₀: There are no risk differences between zip codes
# he we will use the same tests as above but for PostalCode

# Chi-squared for Claim Frequency by Zip
freq_zip = pd.crosstab(df['PostalCode'], df['HasClaim'])
chi2_zip, p_zip, _, _ = stats.chi2_contingency(freq_zip)
print(f"Chi-squared test for Claim Frequency by Zip Code: p-value = {p_zip:.4f}")

# Kruskal-Wallis for Severity
grouped_zip_severity = [group['TotalClaims'].dropna() for _, group in claim_data.groupby('PostalCode')]
h_zip, p_zip_sev = stats.kruskal(*grouped_zip_severity)
print(f"Kruskal-Wallis for Claim Severity by Zip Code: p-value = {p_zip_sev:.4f}")

Chi-squared test for Claim Frequency by Zip Code: p-value = 0.0000
Kruskal-Wallis for Claim Severity by Zip Code: p-value = 0.0000


In [9]:
# Null Hypothesis 3
# H₀: There are no margin differences between zip codes
# We'll use ANOVA or Kruskal-Wallis again.

grouped_margin = [group['Margin'].dropna() for _, group in df.groupby('PostalCode')]
h_margin, p_margin = stats.kruskal(*grouped_margin)

print(f"Kruskal-Wallis for Margin by Zip Code: p-value = {p_margin:.4f}")

Kruskal-Wallis for Margin by Zip Code: p-value = 0.0000


In [10]:
# Null Hypothesis 4
# H₀: There is no significant risk difference between Women and Men
# Frequency

freq_gender = pd.crosstab(df['Gender'], df['HasClaim'])
chi2_gender, p_gender, _, _ = stats.chi2_contingency(freq_gender)

print(f"Chi-squared for Claim Frequency by Gender: p-value = {p_gender:.4f}")

# Severity
grouped_gender = [group['TotalClaims'].dropna() for _, group in claim_data.groupby('Gender')]
h_gender, p_gender_sev = stats.kruskal(*grouped_gender)

print(f"Kruskal-Wallis for Claim Severity by Gender: p-value = {p_gender_sev:.4f}")

Chi-squared for Claim Frequency by Gender: p-value = 0.0266
Kruskal-Wallis for Claim Severity by Gender: p-value = 0.2081


In [ ]:
"""
******************************************************** Interpretation of results *********************************************************

----------------------------------------------------------Hypothesis 1: Province Risk Differences---------------------------------------------------------
Results:
Claim Frequency: p = 0.0000
Claim Severity: p = 0.0000

Interpretation:
Both p-values are much less than 0.05, so we reject the null hypothesis.
This means claim frequency and severity significantly vary across provinces.

Business Implication:
Some provinces (e.g., Gauteng or KwaZulu-Natal) may have higher risk exposure due to higher traffic, theft, or accident rates. Premiums and risk models should be adjusted regionally to account for this variation in risk.


-----------------------------------------------------------Hypothesis 2: Zip Code Risk Differences---------------------------------------------
Results:
Claim Frequency: p = 0.0000
Claim Severity: p = 0.0000

Interpretation:
Again, both p-values are statistically significant.
We reject the null hypothesis: risk varies significantly by zip code.

Business Implication:
Zip-level pricing and risk segmentation could be highly effective. Consider implementing hyperlocal underwriting strategies, as claims are not evenly distributed geographically even within provinces.


------------------------------------------------------------Hypothesis 3: Margin Differences by Zip Code-------------------------------------------
Result:
Margin (Profit) Variation: p = 0.0000

Interpretation:
Strong statistical evidence that profitability (premium - claim) varies by zip.
We reject the null hypothesis.

Business Implication:
Some areas are more profitable, while others likely suffer losses. This supports revising pricing, marketing, or product offerings based on local profitability analysis.


-----------------------------------------------------------Hypothesis 4: Gender-Based Risk Differences-------------------------------
Results:
Claim Frequency: p = 0.0266
Claim Severity: p = 0.2081

Interpretation:
Claim Frequency: p < 0.05 → We reject the null hypothesis. There's a statistically significant difference in how often different genders file claims.
Claim Severity: p > 0.05 → We fail to reject the null hypothesis. There is no significant difference in claim size between genders.

Business Implication:
While men and women file claims at different frequencies, they tend to incur similar claim amounts. Consider using gender as a risk factor for frequency prediction, but not for severity modeling.
"""